In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

train = tf.keras.utils.image_dataset_from_directory('binar/train', shuffle=False,label_mode='categorical')
val = tf.keras.utils.image_dataset_from_directory('binar/val', shuffle= False,label_mode='categorical')

Found 2275 files belonging to 2 classes.
Found 936 files belonging to 2 classes.


# Кошки против собак 

### VGG16

In [3]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [4]:
base_model = tf.keras.applications.VGG16(include_top=False)
base_model.trainable = False

from tensorflow.keras import layers

inputs = layers.Input(shape = ( 256, 256, 3), name='inputLayer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(2, name='outputLayer')(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

In [5]:
model.summary()

Model: "FeatureExtractionModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 poolingLayer (GlobalAverage  (None, 512)              0         
 Pooling2D)                                                      
                                                                 
 outputLayer (Dense)         (None, 2)                 1026      
                                                                 
 activationLayer (Activation  (None, 2)                0         
 )                                                               
                                                                 
Total params: 14,715,714
Trainable params: 1

In [6]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [7]:
model.fit(train, epochs=5, )

Epoch 1/5
72/72 [==============================] - 664s 9s/step - loss: 5.4793 - accuracy: 0.7727
Epoch 2/5
72/72 [==============================] - 614s 9s/step - loss: 6.2035 - accuracy: 0.6725
Epoch 3/5
72/72 [==============================] - 721s 10s/step - loss: 1.2462 - accuracy: 0.8488
Epoch 4/5
72/72 [==============================] - 693s 10s/step - loss: 0.7121 - accuracy: 0.8949
Epoch 5/5
72/72 [==============================] - 629s 9s/step - loss: 0.4603 - accuracy: 0.9196


In [8]:
pred_train, lbl_train = [], []
for i,k in iter(train):
    
    pred_train = pred_train + list(model.predict(i))
    lbl_train = lbl_train + list(k)
    
pred_train, lbl_train = list(np.argmax(np.array(pred_train),1)), list(np.argmax(np.array(lbl_train),1))

In [9]:
print('Точность на обучающей выборке = ',accuracy_score(lbl_train,pred_train))

Точность на обучающей выборке =  0.9046153846153846


In [10]:
print('confusion_matrix на обучающей выборке = \n',confusion_matrix(lbl_train,pred_train))

confusion_matrix на обучающей выборке = 
 [[ 555  212]
 [   5 1503]]


In [11]:
pred_test, lbl_test = [], []
for i,k in iter(val):
    
    pred_test = pred_test + list(model.predict(i))
    lbl_test = lbl_test + list(k)
    
pred_test, lbl_test = list(np.argmax(np.array(pred_test),1)), list(np.argmax(np.array(lbl_test),1))

In [12]:
print('Точность на тестовой выборке = ',accuracy_score(lbl_test,pred_test))

Точность на тестовой выборке =  0.9006410256410257


In [13]:
print('confusion_matrix на тестовой выборке = \n',confusion_matrix(lbl_test,pred_test))

confusion_matrix на тестовой выборке = 
 [[247  92]
 [  1 596]]


### ResNet 

In [14]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [15]:
base_model = tf.keras.applications.ResNet50(include_top=False)
base_model.trainable = False

from tensorflow.keras import layers

inputs = layers.Input(shape = ( 256, 256, 3), name='inputLayer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(2, name='outputLayer')(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

In [16]:
model.summary()

Model: "FeatureExtractionModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 poolingLayer (GlobalAverage  (None, 2048)             0         
 Pooling2D)                                                      
                                                                 
 outputLayer (Dense)         (None, 2)                 4098      
                                                                 
 activationLayer (Activation  (None, 2)                0         
 )                                                               
                                                                 
Total params: 23,591,810
Trainable params: 4

In [17]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [18]:
model.fit(train, epochs=5, )

Epoch 1/5
72/72 [==============================] - 216s 3s/step - loss: 1.2782 - accuracy: 0.8457
Epoch 2/5
72/72 [==============================] - 216s 3s/step - loss: 2.3887 - accuracy: 0.7319
Epoch 3/5
72/72 [==============================] - 220s 3s/step - loss: 0.2656 - accuracy: 0.9169
Epoch 4/5
72/72 [==============================] - 216s 3s/step - loss: 0.1337 - accuracy: 0.9525
Epoch 5/5
72/72 [==============================] - 213s 3s/step - loss: 0.0638 - accuracy: 0.9763


In [19]:
pred_train, lbl_train = [], []
for i,k in iter(train):
    
    pred_train = pred_train + list(model.predict(i))
    lbl_train = lbl_train + list(k)
    
pred_train, lbl_train = list(np.argmax(np.array(pred_train),1)), list(np.argmax(np.array(lbl_train),1))

In [20]:
print('Точность на обучающей выборке = ',accuracy_score(lbl_train,pred_train))

Точность на обучающей выборке =  0.9828571428571429


In [21]:
print('confusion_matrix на обучающей выборке = \n',confusion_matrix(lbl_train,pred_train))

confusion_matrix на обучающей выборке = 
 [[ 733   34]
 [   5 1503]]


In [22]:
pred_test, lbl_test = [], []
for i,k in iter(val):
    
    pred_test = pred_test + list(model.predict(i))
    lbl_test = lbl_test + list(k)
    
pred_test, lbl_test = list(np.argmax(np.array(pred_test),1)), list(np.argmax(np.array(lbl_test),1))

In [23]:
print('Точность на тестовой выборке = ',accuracy_score(lbl_test,pred_test))

Точность на тестовой выборке =  0.9775641025641025


In [24]:
print('confusion_matrix на тестовой выборке = \n',confusion_matrix(lbl_test,pred_test))

confusion_matrix на тестовой выборке = 
 [[320  19]
 [  2 595]]


#### Выводы 
- Обе модели справились с задачей хорошо
- Переобучение не наблюдается 
- ResNet - модель показала себя лучше

# Породы 

### VGG16

In [25]:
train = tf.keras.utils.image_dataset_from_directory('animals/train', shuffle=False,label_mode='categorical')
val = tf.keras.utils.image_dataset_from_directory('animals/val', shuffle= False,label_mode='categorical')

Found 2348 files belonging to 35 classes.
Found 863 files belonging to 35 classes.


In [26]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [27]:
base_model = tf.keras.applications.VGG16(include_top=False)
base_model.trainable = False

from tensorflow.keras import layers

inputs = layers.Input(shape = ( 256, 256, 3), name='inputLayer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(35, name='outputLayer')(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

In [28]:
model.summary()

Model: "FeatureExtractionModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 poolingLayer (GlobalAverage  (None, 512)              0         
 Pooling2D)                                                      
                                                                 
 outputLayer (Dense)         (None, 35)                17955     
                                                                 
 activationLayer (Activation  (None, 35)               0         
 )                                                               
                                                                 
Total params: 14,732,643
Trainable params: 1

In [29]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [30]:
model.fit(train, epochs=5, )

Epoch 1/5
74/74 [==============================] - 584s 8s/step - loss: 20.1800 - accuracy: 0.0077
Epoch 2/5
74/74 [==============================] - 592s 8s/step - loss: 12.2462 - accuracy: 0.0183
Epoch 3/5
74/74 [==============================] - 591s 8s/step - loss: 10.3838 - accuracy: 0.0311
Epoch 4/5
74/74 [==============================] - 591s 8s/step - loss: 8.6415 - accuracy: 0.0669
Epoch 5/5
74/74 [==============================] - 654s 9s/step - loss: 7.0543 - accuracy: 0.1465


In [31]:
pred_train, lbl_train = [], []
for i,k in iter(train):
    
    pred_train = pred_train + list(model.predict(i))
    lbl_train = lbl_train + list(k)
    
pred_train, lbl_train = list(np.argmax(np.array(pred_train),1)), list(np.argmax(np.array(lbl_train),1))

In [32]:
print('Точность на обучающей выборке = ',accuracy_score(lbl_train,pred_train))

Точность на обучающей выборке =  0.4178023850085179


In [33]:
print('confusion_matrix на обучающей выборке = \n',confusion_matrix(lbl_train,pred_train))

confusion_matrix на обучающей выборке = 
 [[ 0  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ... 15  0  0]
 [ 0  0  0 ...  0 45  3]
 [ 0  0  0 ...  0  5 25]]


In [34]:
pred_test, lbl_test = [], []
for i,k in iter(val):
    
    pred_test = pred_test + list(model.predict(i))
    lbl_test = lbl_test + list(k)
    
pred_test, lbl_test = list(np.argmax(np.array(pred_test),1)), list(np.argmax(np.array(lbl_test),1))

In [35]:
print('Точность на тестовой выборке = ',accuracy_score(lbl_test,pred_test))

Точность на тестовой выборке =  0.35805330243337197


In [36]:
print('confusion_matrix на тестовой выборке = \n',confusion_matrix(lbl_test,pred_test))

confusion_matrix на тестовой выборке = 
 [[ 0  1  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0 14  0]
 [ 0  0  0 ...  0  4  6]]


### ResNet 

In [42]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [43]:
base_model = tf.keras.applications.ResNet50(include_top=False)
base_model.trainable = False

from tensorflow.keras import layers

inputs = layers.Input(shape = ( 256, 256, 3), name='inputLayer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(35, name='outputLayer')(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

In [44]:
model.summary()

Model: "FeatureExtractionModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 poolingLayer (GlobalAverage  (None, 2048)             0         
 Pooling2D)                                                      
                                                                 
 outputLayer (Dense)         (None, 35)                71715     
                                                                 
 activationLayer (Activation  (None, 35)               0         
 )                                                               
                                                                 
Total params: 23,659,427
Trainable params: 7

In [45]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [46]:
model.fit(train, epochs=5, )

Epoch 1/5
74/74 [==============================] - 290s 4s/step - loss: 8.1749 - accuracy: 0.0072
Epoch 2/5
74/74 [==============================] - 261s 4s/step - loss: 4.2019 - accuracy: 0.0336
Epoch 3/5
74/74 [==============================] - 248s 3s/step - loss: 3.8176 - accuracy: 0.0967
Epoch 4/5
74/74 [==============================] - 249s 3s/step - loss: 2.9738 - accuracy: 0.2862
Epoch 5/5
74/74 [==============================] - 249s 3s/step - loss: 2.6741 - accuracy: 0.3722


In [47]:
pred_train, lbl_train = [], []
for i,k in iter(train):
    
    pred_train = pred_train + list(model.predict(i))
    lbl_train = lbl_train + list(k)
    
pred_train, lbl_train = list(np.argmax(np.array(pred_train),1)), list(np.argmax(np.array(lbl_train),1))

In [48]:
print('Точность на обучающей выборке = ',accuracy_score(lbl_train,pred_train))

Точность на обучающей выборке =  0.65587734241908


In [49]:
print('confusion_matrix на обучающей выборке = \n',confusion_matrix(lbl_train,pred_train))

confusion_matrix на обучающей выборке = 
 [[ 0  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0 61  1]
 [ 0  0  0 ...  0  0 52]]


In [50]:
pred_test, lbl_test = [], []
for i,k in iter(val):
    
    pred_test = pred_test + list(model.predict(i))
    lbl_test = lbl_test + list(k)
    
pred_test, lbl_test = list(np.argmax(np.array(pred_test),1)), list(np.argmax(np.array(lbl_test),1))

In [51]:
print('Точность на тестовой выборке = ',accuracy_score(lbl_test,pred_test))

Точность на тестовой выборке =  0.574739281575898


In [52]:
print('confusion_matrix на тестовой выборке = \n',confusion_matrix(lbl_test,pred_test))

confusion_matrix на тестовой выборке = 
 [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0 20  0]
 [ 0  0  0 ...  0  0 15]]


#### Выводы 
- У обоих моделей не наблюдается переобучение 
- ResNet - модель показало себя лучше 